In [1]:
## Make ouryelpreviews.csv frmo review.json
## use ourhelpbusinesses.csv to assist in the reduction
##
## T H I S   R U N S   L O N G...
##
## T H I S   R U N S   L O N G...
##
## T H I S   R U N S   L O N G...
##
## as of 5/23 On Richards local machine,
##     process reads abt 50000 and writes abt 5000 selected review out per minute
##     meaning abt 120mins to read 6,000,000 reviews and write approx 600,000 to 1MM

In [2]:
## Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import types
import re
#import ijson

import os
from datetime import datetime

%matplotlib inline
##

datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2019-05-23 09:33:37'

In [3]:
#setup environment
import os
cwd = os.getcwd()
print("Current Working Directory " , cwd)
os.chdir("C:/Users/Richpat/Documents/@syracuse coursework/@IST 718 Big Data Analytics (Python)/finalproject")
print('\nFiles in directory', os.listdir())
print('\n')
pd.show_versions(as_json=True) # True to shorten/flatten output
print ('\nStart date/time', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Current Working Directory  c:\Users\Richpat\Documents\@syracuse coursework\@IST 718 Big Data Analytics (Python)\finalproject

Files in directory ['.ipynb_checkpoints', '2016_Historical_Business_File_Layout.xlsx', 'b.2.1_41473_project_checkpoint_1_clean.pdf', 'bCatTall.csv', 'business.json', 'BusinessCategoryCount.csv', 'checkin.json', 'Dataset_Challenge_Dataset_Agreement.pdf', 'DBHArizona164RestaurantsHighRisk.csv', 'DBHArizona164RestaurantsLowGT9.csv', 'DBHArizona164RestaurantsLowLT10.csv', 'DBHArizona164RestaurantsMediumGT6.csv', 'DBHArizona164RestaurantsMediumLT7.csv', 'DBHArizona164RestaurantsNARisk.csv', 'DBHNevada164RestaurantsHighRisk.csv', 'DBHNevada164RestaurantsLowRisk.csv', 'DBHNevada164RestaurantsMediumRisk.csv', 'DBHNevada164RestaurantsNARisk.csv', 'DBHRestaurants.csv', 'DescribeYelpData.ipynb', 'DRT_IST718_FinalProject_concatDBH.rmd', 'DRT_IST718_FinalProject_ReadJSON.rmd', 'DRT_Thinking_draft.pptx', 'MakeOurYelpBusinesses.ipynb', 'MakeOurYelpReviews.ipynb', 'ourreviews.c

In [4]:
def reviewDFinit():
    #df = pd.DataFrame(columns=['A','B','C','D','E','F','G'])
    #df = pd.DataFrame(columns=["WchChk", "CoachRow", "GSRRow"
    #                  , "GSRRow", "CoachTransfName", "GSRName"
    #                  , "LD", "LR"
    #                  , "FR", "FP", "FT"])
    df = pd.DataFrame({"review_id":["a"], "user_id":["a"], "business_id":["a"]
                       , "stars": ["a"]
                       , "text":["a"]
                       , "date":["a"]})
#    print(pd.DataFrame.head(df))
#    print(df.dtypes)
#    df['review_id'] = df['review_id'].astype('str')
#    df['user_id'] = df['user_id'].astype('str')
#    df['business_id'] = df['business_id'].astype('str')
#    df['stars'] = df['stars'].astype('str')
#    df['text'] = df['text'].astype('str')
#    df['date'] = df['date'].astype('str')
    df = df.drop(df.index[0])
    return(df)

def reviewDFBuild(rid, uid, bid, st, te, dt):
    """ df existing dataframe
        rid review_id
        uid user_id
        bid business_Id
        st stars
        te text
        dt date (and time)
    """
    # Initialize matrix of zeros
#    print('reviewDFAppend in')
#    print ('Review ID', rid)
#    print ('User ID', uid)
#    print ('Business ID', bid)
#    print ('Stars', st)
#    print ('Text', te)
#    print ('Date', dt)
    df2 = pd.DataFrame({"review_id":[rid], "user_id":[uid], "business_id":[bid], "stars":[st], "text":[te], "date":[dt]})
    return(df2)

import re
def tidyUp (indata):
    #replace \n (newline or LF) char with space
    snippet = re.sub('\s*\n\s*', ' ', str(indata))
    #replace \r (newline or LF) char with space
    snippet = re.sub('\s*\r\s*', ' ', str(snippet))
    #tidy up html snippet, change | to ~
    #snippet = re.sub('\s*\|\s*', '~', snippet)
    return snippet

In [5]:
#load "our businesses"
#

OurBusinessDF = pd.read_csv('ouryelpbusinesses.csv', sep='|', encoding='utf-8')

In [6]:
print('\n',pd.crosstab(OurBusinessDF.IsRestaurants, OurBusinessDF.state, rownames = ['IsRestaurants'], colnames = ['']))
print('\n',pd.crosstab(OurBusinessDF.IsShopping, OurBusinessDF.state, rownames = ['IsShopping'], colnames = ['']))
print('\n',pd.crosstab(OurBusinessDF.IsFood, OurBusinessDF.state, rownames = ['IsFood'], colnames = ['']))
print('\n',pd.crosstab(OurBusinessDF.IsHomeservices, OurBusinessDF.state, rownames = ['IsHomeservices'], colnames = ['']))
print('\n',pd.crosstab(OurBusinessDF.IsBeautyspas, OurBusinessDF.state, rownames = ['IsBeautyspas'], colnames = ['']))


                  AZ    NV
IsRestaurants            
False          8317  5161
True           3398  2246

               AZ    NV
IsShopping            
False       9385  5889
True        2330  1518

            AZ    NV
IsFood             
False   10001  6162
True     1714  1245

                   AZ    NV
IsHomeservices            
False           9265  6128
True            2450  1279

                 AZ    NV
IsBeautyspas            
False         9892  6288
True          1823  1119


In [7]:
#REVIEWS - read on a one by one basis (due to memory constraints)
#for each review, look to see if it is for a business that was selected
#if business exists, then the review is selected 

print ('Review Review Filter Start date/time', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

rNoReviewCnt=0
rNoSelected=0
reviewDF=reviewDFinit()
reviewDF.to_csv('ouryelpreviews.csv', sep='|', header='column_names',encoding='utf-8') #create file, write with |, empty set
with open('review.json', encoding='utf-8') as json_file:      
    for data in json_file:
        rNoReviewCnt+=1
        #print ('Type data', type(list(data)), 'len data', len(data))
        #print (data)
        #review=list(data)
        #review=list(map(json.load, data))
        review=json.loads(data)
        businessDFtmp=OurBusinessDF[(OurBusinessDF['business_id']==review['business_id'])]
        if businessDFtmp.shape[0] !=0:
            rNoSelected+=1
            zrid=review['review_id']
            zuid=review['user_id']
            zbid=review['business_id']
            zst=review['stars']
            zte=tidyUp(review['text'])
            zdt=review['date']
            #print ("\nReview number", rNoReviewCnt)
            #print ('Review ID', zrid, 'User ID', zuid, 'Business ID', zbid, 'Stars', zst, 'Date', zdt)
            #print ('Text', zte)
            #print (businessDFtmp)
            #print ('Review data')
            #print (json.dumps(review, indent=4))
            reviewDF=reviewDF.append(reviewDFBuild(zrid, zuid, zbid, zst, zte, zdt))
            if 0 == (rNoSelected % 5000):
                print('Number of reviews counted:', rNoReviewCnt
                      , 'Number of reviews selected:', rNoSelected
                     , 'at date/time', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
                reviewDF.to_csv('ouryelpreviews.csv', sep='|', mode='a', header=False ,encoding='utf-8') #write buffer
                reviewDF=reviewDFinit() # initialize df
            if rNoSelected>7000000:
                break

reviewDF.to_csv('ouryelpreviews.csv', sep='|', mode='a', header=False ,encoding='utf-8') #flush buffer
print('Number of reviews counted:', rNoReviewCnt)
print('Number of reviews selected:', rNoSelected)
print ('Review Review Filter End date/time', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Review Review Filter Start date/time 2019-05-23 09:33:38
Number of reviews counted: 33753 Number of reviews selected: 5000 at date/time 2019-05-23 09:34:38
Number of reviews counted: 67064 Number of reviews selected: 10000 at date/time 2019-05-23 09:35:37
Number of reviews counted: 100518 Number of reviews selected: 15000 at date/time 2019-05-23 09:36:38
Number of reviews counted: 133685 Number of reviews selected: 20000 at date/time 2019-05-23 09:37:36
Number of reviews counted: 166564 Number of reviews selected: 25000 at date/time 2019-05-23 09:38:34
Number of reviews counted: 199029 Number of reviews selected: 30000 at date/time 2019-05-23 09:39:33
Number of reviews counted: 232075 Number of reviews selected: 35000 at date/time 2019-05-23 09:40:34
Number of reviews counted: 264589 Number of reviews selected: 40000 at date/time 2019-05-23 09:41:34
Number of reviews counted: 297700 Number of reviews selected: 45000 at date/time 2019-05-23 09:42:34
Number of reviews counted: 330922 Num

Number of reviews counted: 2540888 Number of reviews selected: 405000 at date/time 2019-05-23 10:50:09
Number of reviews counted: 2571521 Number of reviews selected: 410000 at date/time 2019-05-23 10:51:04
Number of reviews counted: 2603598 Number of reviews selected: 415000 at date/time 2019-05-23 10:52:01
Number of reviews counted: 2634648 Number of reviews selected: 420000 at date/time 2019-05-23 10:52:56
Number of reviews counted: 2666020 Number of reviews selected: 425000 at date/time 2019-05-23 10:53:53
Number of reviews counted: 2694376 Number of reviews selected: 430000 at date/time 2019-05-23 10:54:45
Number of reviews counted: 2723156 Number of reviews selected: 435000 at date/time 2019-05-23 10:55:37
Number of reviews counted: 2751308 Number of reviews selected: 440000 at date/time 2019-05-23 10:56:28
Number of reviews counted: 2779475 Number of reviews selected: 445000 at date/time 2019-05-23 10:57:20
Number of reviews counted: 2807509 Number of reviews selected: 450000 at 

Number of reviews counted: 5072489 Number of reviews selected: 805000 at date/time 2019-05-23 12:06:50
Number of reviews counted: 5102875 Number of reviews selected: 810000 at date/time 2019-05-23 12:07:45
Number of reviews counted: 5134057 Number of reviews selected: 815000 at date/time 2019-05-23 12:08:41
Number of reviews counted: 5164204 Number of reviews selected: 820000 at date/time 2019-05-23 12:09:36
Number of reviews counted: 5194550 Number of reviews selected: 825000 at date/time 2019-05-23 12:10:31
Number of reviews counted: 5225216 Number of reviews selected: 830000 at date/time 2019-05-23 12:11:27
Number of reviews counted: 5255790 Number of reviews selected: 835000 at date/time 2019-05-23 12:12:24
Number of reviews counted: 5287316 Number of reviews selected: 840000 at date/time 2019-05-23 12:13:23
Number of reviews counted: 5319084 Number of reviews selected: 845000 at date/time 2019-05-23 12:14:23
Number of reviews counted: 5351947 Number of reviews selected: 850000 at 